In [1]:
import requests
import json
import pandas as pd
import numpy as np
from api_keys import bls_api_key
from us_states import statesData
from us_counties import usCounties
import calendar
from datetime import datetime


from pymongo import MongoClient
from pprint import pprint

In [2]:
# keys
bls_key = bls_api_key

In [3]:
# Path to the csv containing the confirmed covid cases
county_covid_path = 'Resources/time_series_covid19_confirmed_US.csv'

# Read the csv
county_covid_df = pd.read_csv(county_covid_path)

# Display the first 5 rows
county_covid_df.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,19732,19759,19759,19759,19759,19759,19759,19759,19790,19790
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,69641,69767,69767,69767,69767,69767,69767,69767,69860,69860
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,7451,7474,7474,7474,7474,7474,7474,7474,7485,7485
3,84001007,US,USA,840,1007.0,Bibb,Alabama,US,32.996421,-87.125115,...,8067,8087,8087,8087,8087,8087,8087,8087,8091,8091
4,84001009,US,USA,840,1009.0,Blount,Alabama,US,33.982109,-86.567906,...,18616,18673,18673,18673,18673,18673,18673,18673,18704,18704


In [4]:
# function to generate the last day of every month starting in 2020 and ending in March of 2023 (when they stopped recording covid data)
def generate_last_days(start_year, start_month, end_year, end_month):
    dates = []
    current_year = start_year
    current_month = start_month

    while (current_year < end_year) or (current_year == end_year and current_month <= end_month):
        last_day = calendar.monthrange(current_year, current_month)[1]
        date = datetime(current_year, current_month, last_day).strftime('%-m/%-d/%y')
        dates.append(date)

        if current_month == 12:
            current_month = 1
            current_year += 1
        else:
            current_month += 1

    return dates

start_year = 2020
start_month = 1
end_year = 2023
end_month = 3

last_days = generate_last_days(start_year, start_month, end_year, end_month)

# Display the list of dates
last_days

['1/31/20',
 '2/29/20',
 '3/31/20',
 '4/30/20',
 '5/31/20',
 '6/30/20',
 '7/31/20',
 '8/31/20',
 '9/30/20',
 '10/31/20',
 '11/30/20',
 '12/31/20',
 '1/31/21',
 '2/28/21',
 '3/31/21',
 '4/30/21',
 '5/31/21',
 '6/30/21',
 '7/31/21',
 '8/31/21',
 '9/30/21',
 '10/31/21',
 '11/30/21',
 '12/31/21',
 '1/31/22',
 '2/28/22',
 '3/31/22',
 '4/30/22',
 '5/31/22',
 '6/30/22',
 '7/31/22',
 '8/31/22',
 '9/30/22',
 '10/31/22',
 '11/30/22',
 '12/31/22',
 '1/31/23',
 '2/28/23',
 '3/31/23']

In [5]:
# Using the dates from above, filter the county covid dataframe down to the monthly numbers
county_covid_monthly_df = county_covid_df[['Admin2','Province_State','1/31/20', '2/29/20', '3/31/20', '4/30/20', '5/31/20', '6/30/20', '7/31/20', '8/31/20', '9/30/20', '10/31/20', '11/30/20', '12/31/20',
 '1/31/21', '2/28/21', '3/31/21', '4/30/21', '5/31/21', '6/30/21', '7/31/21', '8/31/21', '9/30/21', '10/31/21', '11/30/21', '12/31/21',
 '1/31/22', '2/28/22', '3/31/22', '4/30/22', '5/31/22', '6/30/22', '7/31/22', '8/31/22', '9/30/22', '10/31/22', '11/30/22', '12/31/22',
 '1/31/23', '2/28/23']]

# Display the first 5 rows
county_covid_monthly_df.head()

,Admin2,Province_State,1/31/20,2/29/20,3/31/20,4/30/20,5/31/20,6/30/20,7/31/20,8/31/20,...,5/31/22,6/30/22,7/31/22,8/31/22,9/30/22,10/31/22,11/30/22,12/31/22,1/31/23,2/28/23
0,Autauga,Alabama,0,0,8,42,227,554,1042,1440,...,15963,16536,17468,17991,18396,18511,18680,18961,19471,19732
1,Baldwin,Alabama,0,0,23,180,298,691,3116,4538,...,56512,58905,61967,64095,65653,65973,66730,67496,68983,69641
2,Barbour,Alabama,0,0,0,38,172,328,603,759,...,5707,5891,6308,6650,6896,6930,6980,7027,7299,7451
3,Bibb,Alabama,0,0,3,40,77,176,369,558,...,6500,6704,6947,7291,7526,7575,7637,7692,7919,8067
4,Blount,Alabama,0,0,7,39,66,218,799,1332,...,15073,15370,15998,16496,16912,17320,17500,17731,18255,18616


In [6]:
# Create a dataframe grouped by state to get the totals per month of each state
state_covid_monthly_df = county_covid_monthly_df.groupby('Province_State').sum().drop(columns=['Admin2'])

# Display the first 5 rows
state_covid_monthly_df.head()

,1/31/20,2/29/20,3/31/20,4/30/20,5/31/20,6/30/20,7/31/20,8/31/20,9/30/20,10/31/20,...,5/31/22,6/30/22,7/31/22,8/31/22,9/30/22,10/31/22,11/30/22,12/31/22,1/31/23,2/28/23
Province_State,,,,,,,,,,,,,,,,,,,,,
Alabama,0,0,1063,7187,18200,38477,88266,126500,154772,193613,...,1314963,1357266,1424411,1479605,1522135,1534287,1549285,1568934,1610535,1638348
Alaska,0,0,129,360,484,1142,3716,6172,8846,16365,...,260059,272415,283643,294498,298178,300177,299869,301432,304368,307073
American Samoa,0,0,0,0,0,0,0,0,0,0,...,6145,6414,7471,8168,8247,8257,8263,8266,8320,8320
Arizona,1,1,1289,7655,19936,79228,174010,201835,218507,245946,...,2049627,2125567,2196429,2254374,2271560,2287886,2337547,2378334,2398200,2434631
Arkansas,0,0,523,3281,7253,20777,42511,61224,83697,112190,...,843362,865592,903931,932639,951644,959855,968871,985493,996758,1004752


In [7]:
# Calculate the difference between adjacent columns starting from 1/31/20
start_col = '1/31/20'
start_idx = state_covid_monthly_df.columns.get_loc(start_col)

state_covid_monthly_df_diff = state_covid_monthly_df.iloc[:, start_idx:].diff(axis=1)

# Fill NaNs with 0 
state_covid_monthly_df_diff = state_covid_monthly_df_diff.fillna(0)

# Display dataframe
state_covid_monthly_df_diff.head()

,1/31/20,2/29/20,3/31/20,4/30/20,5/31/20,6/30/20,7/31/20,8/31/20,9/30/20,10/31/20,...,5/31/22,6/30/22,7/31/22,8/31/22,9/30/22,10/31/22,11/30/22,12/31/22,1/31/23,2/28/23
Province_State,,,,,,,,,,,,,,,,,,,,,
Alabama,0.0,0,1063,6124,11013,20277,49789,38234,28272,38841,...,14230,42303,67145,55194,42530,12152,14998,19649,41601,27813
Alaska,0.0,0,129,231,124,658,2574,2456,2674,7519,...,7057,12356,11228,10855,3680,1999,-308,1563,2936,2705
American Samoa,0.0,0,0,0,0,0,0,0,0,0,...,323,269,1057,697,79,10,6,3,54,0
Arizona,0.0,0,1288,6366,12281,59292,94782,27825,16672,27439,...,28103,75940,70862,57945,17186,16326,49661,40787,19866,36431
Arkansas,0.0,0,523,2758,3972,13524,21734,18713,22473,28493,...,7505,22230,38339,28708,19005,8211,9016,16622,11265,7994


In [8]:
# Path to the csv containing the covid case deaths
county_covid_deaths_path = 'Resources/time_series_covid19_deaths_US.csv'

# Read the csv
county_covid_deaths_df = pd.read_csv(county_covid_deaths_path)

# Display the first 5 rows
county_covid_deaths_df.head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,2/28/23,3/1/23,3/2/23,3/3/23,3/4/23,3/5/23,3/6/23,3/7/23,3/8/23,3/9/23
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,230,232,232,232,232,232,232,232,232,232
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,724,726,726,726,726,726,726,726,727,727
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,103,103,103,103,103,103,103,103,103,103
3,84001007,US,USA,840,1007.0,Bibb,Alabama,US,32.996421,-87.125115,...,109,109,109,109,109,109,109,109,109,109
4,84001009,US,USA,840,1009.0,Blount,Alabama,US,33.982109,-86.567906,...,261,261,261,261,261,261,261,261,261,261


In [9]:
# Using the dates from above, filter the county covid deaths dataframe down to the monthly numbers
county_covid_deaths_monthly_df = county_covid_deaths_df[['Admin2','Province_State','1/31/20', '2/29/20', '3/31/20', '4/30/20', '5/31/20', '6/30/20', '7/31/20', '8/31/20', '9/30/20', '10/31/20', '11/30/20', '12/31/20',
 '1/31/21', '2/28/21', '3/31/21', '4/30/21', '5/31/21', '6/30/21', '7/31/21', '8/31/21', '9/30/21', '10/31/21', '11/30/21', '12/31/21',
 '1/31/22', '2/28/22', '3/31/22', '4/30/22', '5/31/22', '6/30/22', '7/31/22', '8/31/22', '9/30/22', '10/31/22', '11/30/22', '12/31/22',
 '1/31/23', '2/28/23']]

# Display the first 5 rows
county_covid_deaths_monthly_df.head()

,Admin2,Province_State,1/31/20,2/29/20,3/31/20,4/30/20,5/31/20,6/30/20,7/31/20,8/31/20,...,5/31/22,6/30/22,7/31/22,8/31/22,9/30/22,10/31/22,11/30/22,12/31/22,1/31/23,2/28/23
0,Autauga,Alabama,0,0,0,4,4,11,20,22,...,216,217,220,222,227,228,230,230,230,230
1,Baldwin,Alabama,0,0,1,3,9,9,21,38,...,683,683,687,693,712,716,716,719,723,724
2,Barbour,Alabama,0,0,0,1,1,1,5,7,...,99,99,100,101,103,103,103,103,103,103
3,Bibb,Alabama,0,0,0,0,1,1,2,7,...,105,105,105,105,107,108,108,108,109,109
4,Blount,Alabama,0,0,0,0,1,1,3,11,...,244,246,247,251,258,258,259,260,261,261


In [10]:
# Create a dataframe grouped by state to get the totals per month of each state
state_covid_deaths_monthly_df = county_covid_deaths_monthly_df.groupby('Province_State').sum().drop(columns=['Admin2'])

# Display the first 5 rows
state_covid_deaths_monthly_df.head()

,1/31/20,2/29/20,3/31/20,4/30/20,5/31/20,6/30/20,7/31/20,8/31/20,9/30/20,10/31/20,...,5/31/22,6/30/22,7/31/22,8/31/22,9/30/22,10/31/22,11/30/22,12/31/22,1/31/23,2/28/23
Province_State,,,,,,,,,,,,,,,,,,,,,
Alabama,0,0,23,272,630,950,1580,2182,2540,2967,...,19664,19755,19891,20048,20422,20558,20652,20737,20870,20932
Alaska,0,0,3,9,10,14,23,37,56,82,...,1285,1285,1309,1338,1364,1413,1436,1455,1473,1486
American Samoa,0,0,0,0,0,0,0,0,0,0,...,31,31,33,34,34,34,34,34,34,34
Arizona,0,0,25,320,907,1645,3694,5029,5650,5979,...,30299,30515,30768,31114,31370,31548,31751,32182,32775,33042
Arkansas,0,0,8,61,133,270,453,797,1369,1925,...,11484,11581,11719,11918,12165,12489,12564,12695,12852,12980


In [11]:
# Calculate the difference between adjacent columns starting from 1/31/20
start_col = '1/31/20'
start_idx = state_covid_deaths_monthly_df.columns.get_loc(start_col)

state_covid_deaths_monthly_df_diff = state_covid_deaths_monthly_df.iloc[:, start_idx:].diff(axis=1)

# Fill NaNs with 0 
state_covid_deaths_monthly_df_diff = state_covid_deaths_monthly_df_diff.fillna(0)

# Display dataframe
state_covid_deaths_monthly_df_diff.head()

,1/31/20,2/29/20,3/31/20,4/30/20,5/31/20,6/30/20,7/31/20,8/31/20,9/30/20,10/31/20,...,5/31/22,6/30/22,7/31/22,8/31/22,9/30/22,10/31/22,11/30/22,12/31/22,1/31/23,2/28/23
Province_State,,,,,,,,,,,,,,,,,,,,,
Alabama,0.0,0,23,249,358,320,630,602,358,427,...,97,91,136,157,374,136,94,85,133,62
Alaska,0.0,0,3,6,1,4,9,14,19,26,...,33,0,24,29,26,49,23,19,18,13
American Samoa,0.0,0,0,0,0,0,0,0,0,0,...,5,0,2,1,0,0,0,0,0,0
Arizona,0.0,0,25,295,587,738,2049,1335,621,329,...,348,216,253,346,256,178,203,431,593,267
Arkansas,0.0,0,8,53,72,137,183,344,572,556,...,95,97,138,199,247,324,75,131,157,128


In [12]:
# List of series IDs for unemployment in each state
series_ids = [
    'LASST010000000000003', 'LASST020000000000003', 'LASST040000000000003',
    'LASST050000000000003', 'LASST060000000000003', 'LASST080000000000003',
    'LASST090000000000003', 'LASST100000000000003', 'LASST110000000000003','LASST120000000000003',
    'LASST130000000000003', 'LASST150000000000003', 'LASST160000000000003',
    'LASST170000000000003', 'LASST180000000000003', 'LASST190000000000003',
    'LASST200000000000003', 'LASST210000000000003', 'LASST220000000000003',
    'LASST230000000000003', 'LASST240000000000003', 'LASST250000000000003',
    'LASST260000000000003', 'LASST270000000000003', 'LASST280000000000003',
    'LASST290000000000003', 'LASST300000000000003', 'LASST310000000000003',
    'LASST320000000000003', 'LASST330000000000003', 'LASST340000000000003',
    'LASST350000000000003', 'LASST360000000000003', 'LASST370000000000003',
    'LASST380000000000003', 'LASST390000000000003', 'LASST400000000000003',
    'LASST410000000000003', 'LASST420000000000003', 'LASST440000000000003',
    'LASST450000000000003', 'LASST460000000000003', 'LASST470000000000003',
    'LASST480000000000003', 'LASST490000000000003', 'LASST500000000000003',
    'LASST510000000000003', 'LASST530000000000003', 'LASST540000000000003',
    'LASST550000000000003', 'LASST560000000000003','LASST720000000000003',
]
# List of states
states = [
    "Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware",'District of Columbia', "Florida", 
    "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", 
    "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", 
    "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", 
    "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", 
    "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming",'Puerto Rico'
]

# Creating the dictionary
state_series_dict_unemployment = dict(zip(states, series_ids))

In [13]:
state_series_dict_unemployment

{'Alabama': 'LASST010000000000003',
 'Alaska': 'LASST020000000000003',
 'Arizona': 'LASST040000000000003',
 'Arkansas': 'LASST050000000000003',
 'California': 'LASST060000000000003',
 'Colorado': 'LASST080000000000003',
 'Connecticut': 'LASST090000000000003',
 'Delaware': 'LASST100000000000003',
 'District of Columbia': 'LASST110000000000003',
 'Florida': 'LASST120000000000003',
 'Georgia': 'LASST130000000000003',
 'Hawaii': 'LASST150000000000003',
 'Idaho': 'LASST160000000000003',
 'Illinois': 'LASST170000000000003',
 'Indiana': 'LASST180000000000003',
 'Iowa': 'LASST190000000000003',
 'Kansas': 'LASST200000000000003',
 'Kentucky': 'LASST210000000000003',
 'Louisiana': 'LASST220000000000003',
 'Maine': 'LASST230000000000003',
 'Maryland': 'LASST240000000000003',
 'Massachusetts': 'LASST250000000000003',
 'Michigan': 'LASST260000000000003',
 'Minnesota': 'LASST270000000000003',
 'Mississippi': 'LASST280000000000003',
 'Missouri': 'LASST290000000000003',
 'Montana': 'LASST30000000000000

In [14]:
# Create the payload for the api call
payload = {
    "seriesid": series_ids,
    "startyear": "2020",
    "endyear": "2023",
    "registrationkey": bls_api_key
}

In [15]:
# Get our url for the api call
url = "https://api.bls.gov/publicAPI/v2/timeseries/data/"

In [16]:
# Make our api call to get the unemployment rates for each state
response = requests.post(url, data=json.dumps(payload), headers={'Content-Type': 'application/json'})
response

<Response [200]>

In [17]:
# Convert our response to a json format
json_response = response.json()
json_response

{'status': 'REQUEST_SUCCEEDED',
 'responseTime': 17706,
 'message': ['Requested Series list has been reduced to the system-allowed limit of 50 series.'],
 'Results': {'series': [{'seriesID': 'LASST010000000000003',
    'data': [{'year': '2023',
      'period': 'M12',
      'periodName': 'December',
      'value': '2.8',
      'footnotes': [{'code': 'R',
        'text': 'Data were subject to revision on March 1, 2024.'}]},
     {'year': '2023',
      'period': 'M11',
      'periodName': 'November',
      'value': '2.8',
      'footnotes': [{'code': 'R',
        'text': 'Data were subject to revision on March 1, 2024.'}]},
     {'year': '2023',
      'period': 'M10',
      'periodName': 'October',
      'value': '2.8',
      'footnotes': [{'code': 'R',
        'text': 'Data were subject to revision on March 1, 2024.'}]},
     {'year': '2023',
      'period': 'M09',
      'periodName': 'September',
      'value': '2.7',
      'footnotes': [{'code': 'R',
        'text': 'Data were subject 

In [18]:
# Grab the series data on unemployment from our json response
unemployment_series_list = json_response['Results']['series']
unemployment_series_list

[{'seriesID': 'LASST010000000000003',
  'data': [{'year': '2023',
    'period': 'M12',
    'periodName': 'December',
    'value': '2.8',
    'footnotes': [{'code': 'R',
      'text': 'Data were subject to revision on March 1, 2024.'}]},
   {'year': '2023',
    'period': 'M11',
    'periodName': 'November',
    'value': '2.8',
    'footnotes': [{'code': 'R',
      'text': 'Data were subject to revision on March 1, 2024.'}]},
   {'year': '2023',
    'period': 'M10',
    'periodName': 'October',
    'value': '2.8',
    'footnotes': [{'code': 'R',
      'text': 'Data were subject to revision on March 1, 2024.'}]},
   {'year': '2023',
    'period': 'M09',
    'periodName': 'September',
    'value': '2.7',
    'footnotes': [{'code': 'R',
      'text': 'Data were subject to revision on March 1, 2024.'}]},
   {'year': '2023',
    'period': 'M08',
    'periodName': 'August',
    'value': '2.5',
    'footnotes': [{'code': 'R',
      'text': 'Data were subject to revision on March 1, 2024.'}]},
 

In [19]:
# Import population data for our states collection in the database
population_df = pd.read_csv('./Resources/NST-EST2023-ALLDATA.csv')
population_df.head(20)

,SUMLEV,REGION,DIVISION,STATE,NAME,ESTIMATESBASE2020,POPESTIMATE2020,POPESTIMATE2021,POPESTIMATE2022,POPESTIMATE2023,...,RNATURALCHG2023,RINTERNATIONALMIG2021,RINTERNATIONALMIG2022,RINTERNATIONALMIG2023,RDOMESTICMIG2021,RDOMESTICMIG2022,RDOMESTICMIG2023,RNETMIG2021,RNETMIG2022,RNETMIG2023
0,10,0,0,0,United States,331464948,331526933,332048977,333271411,334914895,...,1.510043,1.133278,3.003867,3.409196,0.000000,0.000000,0.000000,1.133278,3.003867,3.409196
1,20,1,0,0,Northeast Region,57614141,57430477,57243423,57026847,56983517,...,0.976280,1.403074,3.678035,3.947167,-4.823207,-7.881683,-5.671414,-3.420133,-4.203648,-1.724247
2,30,1,1,0,New England,15119994,15057898,15106108,15120739,15159777,...,-0.019551,1.772643,4.532659,5.024221,1.615767,-3.086263,-2.445863,3.388409,1.446396,2.578358
3,30,1,2,0,Middle Atlantic,42494147,42372579,42137315,41906108,41823740,...,1.336417,1.271165,3.370662,3.557656,-7.121462,-9.606391,-6.837920,-5.850297,-6.235729,-3.280264
4,20,2,0,0,Midwest Region,68987296,68969794,68850246,68783028,68909283,...,0.665803,0.802684,2.098649,2.409866,-2.577042,-2.636615,-1.245226,-1.774357,-0.537966,1.164640
5,30,2,3,0,East North Central,47369629,47345074,47187461,47098310,47146039,...,0.267730,0.783328,2.059865,2.608369,-3.730335,-3.042877,-1.867613,-2.947007,-0.983012,0.740755
6,30,2,4,0,West North Central,21617667,21624720,21662785,21684718,21763244,...,1.529278,0.844955,2.183009,1.979287,-0.058446,-1.752950,0.104815,0.786509,0.430059,2.084102
7,20,3,0,0,South Region,126268529,126465281,127353282,128702030,130125290,...,1.679436,1.223063,3.180789,3.862653,5.640927,6.689031,5.457430,6.863990,9.869821,9.320083
8,30,3,5,0,South Atlantic,66091701,66174797,66673601,67445114,68225883,...,0.814574,1.471181,3.796204,4.698587,6.559311,7.723739,5.999602,8.030492,11.519943,10.698189
9,30,3,6,0,East South Central,19402683,19424519,19471271,19573370,19700801,...,-0.200794,0.375568,0.940360,1.571771,3.739016,5.938331,5.118479,4.114584,6.878690,6.690249


In [20]:
population_df = population_df[['NAME','POPESTIMATE2020','POPESTIMATE2021','POPESTIMATE2022','POPESTIMATE2023']]
population_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 66 entries, 0 to 65
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   NAME             66 non-null     object
 1   POPESTIMATE2020  66 non-null     int64 
 2   POPESTIMATE2021  66 non-null     int64 
 3   POPESTIMATE2022  66 non-null     int64 
 4   POPESTIMATE2023  66 non-null     int64 
dtypes: int64(4), object(1)
memory usage: 2.7+ KB


In [21]:
# Create our list of states data for the states collection
states_list = statesData['features']
states_list

[{'type': 'Feature',
  'id': '01',
  'properties': {'name': 'Alabama', 'density': 94.65},
  'geometry': {'type': 'Polygon',
   'coordinates': [[[-87.359296, 35.00118],
     [-85.606675, 34.984749],
     [-85.431413, 34.124869],
     [-85.184951, 32.859696],
     [-85.069935, 32.580372],
     [-84.960397, 32.421541],
     [-85.004212, 32.322956],
     [-84.889196, 32.262709],
     [-85.058981, 32.13674],
     [-85.053504, 32.01077],
     [-85.141136, 31.840985],
     [-85.042551, 31.539753],
     [-85.113751, 31.27686],
     [-85.004212, 31.003013],
     [-85.497137, 30.997536],
     [-87.600282, 30.997536],
     [-87.633143, 30.86609],
     [-87.408589, 30.674397],
     [-87.446927, 30.510088],
     [-87.37025, 30.427934],
     [-87.518128, 30.280057],
     [-87.655051, 30.247195],
     [-87.90699, 30.411504],
     [-87.934375, 30.657966],
     [-88.011052, 30.685351],
     [-88.10416, 30.499135],
     [-88.137022, 30.318396],
     [-88.394438, 30.367688],
     [-88.471115, 31.895754],

In [22]:
list(state_covid_deaths_monthly_df_diff.loc['Ohio'].loc['1/31/20':'12/31/20'].astype(int))

[0, 0, 90, 992, 1051, 467, 608, 704, 558, 854, 2759, 5537]

In [23]:
# Prepare our data for the states collection in our database
states_list = statesData['features']
for state in states_list:
    dic = state['properties']
    state_name = state['properties']['name']
    dic['Population']=[
        {
            'year':2020,
            'Population':int(population_df.loc[population_df['NAME']==state_name,'POPESTIMATE2020'].values[0])
        },
        {
            'year':2021,
            'Population':int(population_df.loc[population_df['NAME']==state_name,'POPESTIMATE2021'].values[0])
        },
        {
            'year':2022,
            'Population':int(population_df.loc[population_df['NAME']==state_name,'POPESTIMATE2022'].values[0])
        },
        {
            'year':2023,
            'Population':int(population_df.loc[population_df['NAME']==state_name,'POPESTIMATE2023'].values[0])
        }]
    state_unemployment_id = state_series_dict_unemployment[state_name]
    for item in unemployment_series_list:
        if item['seriesID'] == state_unemployment_id:
            dic['Unemployment Data'] = item['data']
            for thing in dic['Unemployment Data']:
                thing['year']=int(thing['year'])
                thing['value']=float(thing['value'])
    dic['Covid Confirmed'] = [
        {
            'year':2020,
            'New Cases':list(state_covid_monthly_df_diff.loc[state_name].loc['1/31/20':'12/31/20'].astype(int))
        },
        {
            'year':2021,
            'New Cases':list(state_covid_monthly_df_diff.loc[state_name].loc['1/31/21':'12/31/21'].astype(int))
        },
        {
            'year':2022,
            'New Cases':list(state_covid_monthly_df_diff.loc[state_name].loc['1/31/22':'12/31/22'].astype(int))
        },
        {
            'year':2023,
            'New Cases':list(state_covid_monthly_df_diff.loc[state_name].loc['1/31/23':'2/28/23'].astype(int))
        }]
    dic['Covid Deaths'] = [
        {
            'year':2020,
            'New Deaths':list(state_covid_deaths_monthly_df_diff.loc[state_name].loc['1/31/20':'12/31/20'].astype(int))
        },
        {
            'year':2021,
            'New Deaths':list(state_covid_deaths_monthly_df_diff.loc[state_name].loc['1/31/21':'12/31/21'].astype(int))
        },
        {
            'year':2022,
            'New Deaths':list(state_covid_deaths_monthly_df_diff.loc[state_name].loc['1/31/22':'12/31/22'].astype(int))
        },
        {
            'year':2023,
            'New Deaths':list(state_covid_deaths_monthly_df_diff.loc[state_name].loc['1/31/23':'2/28/23'].astype(int))
        }]
    state['properties']=dic
states_list

[{'type': 'Feature',
  'id': '01',
  'properties': {'name': 'Alabama',
   'density': 94.65,
   'Population': [{'year': 2020, 'Population': 5031864},
    {'year': 2021, 'Population': 5050380},
    {'year': 2022, 'Population': 5073903},
    {'year': 2023, 'Population': 5108468}],
   'Unemployment Data': [{'year': 2023,
     'period': 'M12',
     'periodName': 'December',
     'value': 2.8,
     'footnotes': [{'code': 'R',
       'text': 'Data were subject to revision on March 1, 2024.'}]},
    {'year': 2023,
     'period': 'M11',
     'periodName': 'November',
     'value': 2.8,
     'footnotes': [{'code': 'R',
       'text': 'Data were subject to revision on March 1, 2024.'}]},
    {'year': 2023,
     'period': 'M10',
     'periodName': 'October',
     'value': 2.8,
     'footnotes': [{'code': 'R',
       'text': 'Data were subject to revision on March 1, 2024.'}]},
    {'year': 2023,
     'period': 'M09',
     'periodName': 'September',
     'value': 2.7,
     'footnotes': [{'code': 'R

In [24]:
# Bring in county data for the counties collection in our database
counties_data = usCounties['features']

In [25]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [26]:
# Check for our database
mongo.list_database_names()

['admin',
 'autosaurus',
 'classDB',
 'config',
 'epa',
 'fruit_db',
 'local',
 'met',
 'school',
 'travel_db',
 'uk_food',
 'us_states_db']

In [27]:
# Connect with the us_states_db
states_db = mongo['us_states_db']

# Connect with each collection in our database
states_collection = states_db['states']
counties_collection = states_db['counties']
series_ids = states_db['series_ids']


In [28]:
# Insert the state data into the states collection
states_collection.insert_many(states_list)

# Insert the county data into the counties collection
counties_collection.insert_many(counties_data)

# Insert the unemployment series ids into the series_ids collection
unemployment_series_ids = {
    'Unemployment Series Ids': state_series_dict_unemployment
}
series_ids.insert_one(unemployment_series_ids)

InsertOneResult(ObjectId('667015652423e5c9d587c261'), acknowledged=True)

In [29]:
states_collection.find_one()

{'_id': ObjectId('667015642423e5c9d587b598'),
 'type': 'Feature',
 'id': '01',
 'properties': {'name': 'Alabama',
  'density': 94.65,
  'Population': [{'year': 2020, 'Population': 5031864},
   {'year': 2021, 'Population': 5050380},
   {'year': 2022, 'Population': 5073903},
   {'year': 2023, 'Population': 5108468}],
  'Unemployment Data': [{'year': 2023,
    'period': 'M12',
    'periodName': 'December',
    'value': 2.8,
    'footnotes': [{'code': 'R',
      'text': 'Data were subject to revision on March 1, 2024.'}]},
   {'year': 2023,
    'period': 'M11',
    'periodName': 'November',
    'value': 2.8,
    'footnotes': [{'code': 'R',
      'text': 'Data were subject to revision on March 1, 2024.'}]},
   {'year': 2023,
    'period': 'M10',
    'periodName': 'October',
    'value': 2.8,
    'footnotes': [{'code': 'R',
      'text': 'Data were subject to revision on March 1, 2024.'}]},
   {'year': 2023,
    'period': 'M09',
    'periodName': 'September',
    'value': 2.7,
    'footnotes

In [30]:
counties_collection.find_one()

{'_id': ObjectId('667015642423e5c9d587b5cc'),
 'type': 'Feature',
 'properties': {'GEO_ID': '0500000US02261',
  'STATE': '02',
  'COUNTY': '261',
  'NAME': 'Valdez-Cordova',
  'LSAD': 'CA',
  'CENSUSAREA': 34239.88},
 'geometry': {'type': 'MultiPolygon',
  'coordinates': [[[[-147.483828, 60.618636],
     [-147.500009, 60.653852],
     [-147.483828, 60.683358],
     [-147.487635, 60.728092],
     [-147.395312, 60.74332],
     [-147.383891, 60.741417],
     [-147.362, 60.714767],
     [-147.3087, 60.665274],
     [-147.348675, 60.627202],
     [-147.454323, 60.619588],
     [-147.483828, 60.618636]]],
   [[[-147.341061, 60.305499],
     [-147.340109, 60.275042],
     [-147.483828, 60.224598],
     [-147.499057, 60.235067],
     [-147.505719, 60.253151],
     [-147.496201, 60.265524],
     [-147.421962, 60.279801],
     [-147.341061, 60.305499]]],
   [[[-147.217704, 60.293504],
     [-147.19494, 60.304563],
     [-147.183277, 60.32068],
     [-147.185243, 60.323083],
     [-147.195608, 60

In [31]:
series_ids.find_one()

{'_id': ObjectId('667015652423e5c9d587c261'),
 'Unemployment Series Ids': {'Alabama': 'LASST010000000000003',
  'Alaska': 'LASST020000000000003',
  'Arizona': 'LASST040000000000003',
  'Arkansas': 'LASST050000000000003',
  'California': 'LASST060000000000003',
  'Colorado': 'LASST080000000000003',
  'Connecticut': 'LASST090000000000003',
  'Delaware': 'LASST100000000000003',
  'District of Columbia': 'LASST110000000000003',
  'Florida': 'LASST120000000000003',
  'Georgia': 'LASST130000000000003',
  'Hawaii': 'LASST150000000000003',
  'Idaho': 'LASST160000000000003',
  'Illinois': 'LASST170000000000003',
  'Indiana': 'LASST180000000000003',
  'Iowa': 'LASST190000000000003',
  'Kansas': 'LASST200000000000003',
  'Kentucky': 'LASST210000000000003',
  'Louisiana': 'LASST220000000000003',
  'Maine': 'LASST230000000000003',
  'Maryland': 'LASST240000000000003',
  'Massachusetts': 'LASST250000000000003',
  'Michigan': 'LASST260000000000003',
  'Minnesota': 'LASST270000000000003',
  'Mississip